<a href="https://colab.research.google.com/github/JuanOspina3/Proyecto-UDEA-ai4eng-20251---Pruebas-Saber-Pro-Colombia/blob/main/03%20-%20svm_onehot_standard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 📦 Importación de librerías
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 📂 Carga de datos
train = pd.read_csv(r"C:\Users\juan2\OneDrive\Escritorio\train.csv")
test = pd.read_csv(r"C:\Users\juan2\OneDrive\Escritorio\test.csv")

X = train.drop(columns=["RENDIMIENTO_GLOBAL"])
y = train["RENDIMIENTO_GLOBAL"]
test_ids = test["ID"]

# 🔍 Columnas categóricas y numéricas
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

# ⚙️ Preprocesador
preprocessor = ColumnTransformer([
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ]), num_cols),

    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
    ]), cat_cols)
])

# 🔗 Modelo final con SVM
model = Pipeline([
    ("pre", preprocessor),
    ("clf", LinearSVC(max_iter=10000))
])

# 🧪 División de datos
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, random_state=42)

# 🚀 Entrenamiento y evaluación
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
acc = accuracy_score(y_val, y_pred)
print(f"\n📊 Accuracy en validación con SVM: {acc:.5f}")

# 🏁 Entrenamiento final con todos los datos
model.fit(X, y)
y_test_pred = model.predict(test)

# 💾 Generación del archivo de submission
submission = pd.DataFrame({
    "ID": test_ids,
    "RENDIMIENTO_GLOBAL": y_test_pred
})

submission.to_csv(r"C:\Users\juan2\OneDrive\Escritorio\submission_svm.csv", index=False)
print("\n✅ Archivo submission_svm.csv guardado correctamente.")
